<a href="https://colab.research.google.com/github/davidest1995/personal_projects/blob/main/Optimizaci%C3%B3n%20de%20n%C3%BAmeros%20de%20telefono.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
## Optimización de números de diferentes tablas de una base de datos en POSTGRESQL, dando como resultado una única tabla con numeros de contacto únicos previamente filtrados por cada cédula.

## Validación de un número específico dentro de la tabla llamada "FUENTE_TOTAL".


CREATE OR REPLACE FUNCTION public.is_blocked_number(
	check_number bigint)
    RETURNS boolean
    LANGUAGE 'plpgsql'
    COST 100
    VOLATILE PARALLEL UNSAFE
AS $BODY$
  
Declare  
 numbers integer; 
Begin
SELECT COUNT(*) INTO numbers FROM PUBLIC."EXCLUIR_TELEFONOS" WHERE "TELEFONOS" = check_number;
RETURN numbers > 0;
End;  
$BODY$;

ALTER FUNCTION public.is_blocked_number(bigint)
    OWNER TO postgres;


## Validación de un número específico dentro de la tabla llamada "TABLA_TELEFONOS".

CREATE OR REPLACE FUNCTION public.is_repeated_number(
	check_number bigint)
    RETURNS boolean
    LANGUAGE 'plpgsql'
    COST 100
    VOLATILE PARALLEL UNSAFE
AS $BODY$
  
Declare  
 numbers integer; 
Begin
SELECT COUNT(*) INTO numbers FROM PUBLIC."TABLA_TELEFONOS" WHERE "TELEFONO" = check_number;
RETURN numbers > 0;
End;  
$BODY$;

ALTER FUNCTION public.is_repeated_number(bigint)
    OWNER TO postgres;

## CREACIÓN DE PROCEDURES LOS CUALES IMPLEMENTAN LAS FUNCIONES PARA CREAR UNA TABLA CON REGISTROS NUMERICOS ÚNICOS POR CADA CLIENTE.
-------------------
-- PROCEDURE: public.limpieza_telefonos()

-- DROP PROCEDURE IF EXISTS public.limpieza_telefonos();

CREATE OR REPLACE PROCEDURE public.limpieza_telefonos(
	)
LANGUAGE 'plpgsql'
AS $BODY$
DECLARE
	fila record;
BEGIN

	FOR fila IN 
		SELECT "CC", "TELEFONO1", "TELEFONO2", "TELEFONO3", "TELEFONO4", "TELEFONO5", "TELEFONO6" 
		from PUBLIC."FUENTE_TOTAL"
	LOOP
		CALL insertar_telefonos(fila."CC", fila."TELEFONO1");
		CALL insertar_telefonos(fila."CC", fila."TELEFONO2");
		CALL insertar_telefonos(fila."CC", fila."TELEFONO3");
		CALL insertar_telefonos(fila."CC", fila."TELEFONO4");
		CALL insertar_telefonos(fila."CC", fila."TELEFONO5");
		CALL insertar_telefonos(fila."CC", fila."TELEFONO6");
	END LOOP;

END;
$BODY$;
ALTER PROCEDURE public.limpieza_telefonos()
    OWNER TO postgres;
	
CALL public.limpieza_telefonos();

--------------------------------------
CREATE OR REPLACE PROCEDURE INSERTAR_TELEFONOS(_CC integer , _NUMERO bigint )
LANGUAGE plpgsql
AS $BODY$
BEGIN
	IF (_NUMERO IS NOT NULL) THEN
		IF NOT (SELECT * FROM is_blocked_number(_NUMERO)) THEN
			-- Numero no bloqueado
			raise notice 'NO BLOQUEADO: %', _NUMERO;
			IF NOT (SELECT * FROM is_repeated_number(_NUMERO)) THEN
				-- Numero no repetido
				-- Insertar en la tabla de telefonos
				INSERT INTO PUBLIC."TABLA_TELEFONOS" VALUES(_NUMERO);
				-- Insertar en la tabla de agregacion
				INSERT INTO PUBLIC."TABLA_AGREGACION" VALUES(_CC, _NUMERO);
				raise notice 'Inserta-> Cliente: %  Numero: %', _CC, _NUMERO;
			ELSE
				-- Numero repetido
				-- Se indica en la tabla numero que esta repetido
				UPDATE 
					PUBLIC."TABLA_TELEFONOS" 
					SET "REPETIDO" = TRUE
					WHERE "TELEFONO" = _NUMERO;
				-- se borra el registro con el numero repetido en la tabla de agregacion
				DELETE FROM
					PUBLIC."TABLA_AGREGACION"
					WHERE "NUMERO" = _NUMERO;
					raise notice 'Borra ->  Numero: %', _NUMERO;
			END IF;
		ELSE
			--Numero bloqueado
			raise notice 'BLOQUEADO: %', _NUMERO;
		END IF;
	ELSE
		raise notice 'Filtra nulo';
	END IF;
END;
$BODY$;

-- LLamar la funcion
CALL INSERTAR_TELEFONOS();